# jupyter notebook v7 remediations

this document massages the annotation object model of a running jupyter classic notebook. the goal is to get to near to an idealized representation devised in the notebooks for all project.

in this representation, we add landmarks and start to codify the semantics of a cell area.

click "trust notebook" in the File Menu to execute this code yourself.

out of scope: 
* the split panel handler has an [APG Window Splitter Pattern](https://www.w3.org/WAI/ARIA/apg/patterns/windowsplitter/) that should be implemented.
* tab traps are bad. tab in cell mode should not enter edit mode automatically.
* can't mess with the native find experience, and if you do it better be good.
  * so i'm real worried about .jp-DocumentSearch-overlay

## fixing document level landmarks

In [ ]:
    %%javascript
    document.querySelectorAll("#top-panel").forEach((x,i) =>{x.setAttribute("aria-label", "Notebook Menu");});
    // it is going to make no sense to anyone that the jupyter label sends you to open files
    document.querySelectorAll("#top-panel #jp-NotebookLogo").forEach((x,i) =>{
        x.setAttribute("title", "Open File Browser in a New Tab");
        x.setAttribute("alt", "Open File Browser in a New Tab");
    });
    document.querySelectorAll("#top-panel #jp-title h1").children.forEach((x,i) =>{
        // we can't set our own h1 as that will mess up custom content.
        // the css depends on the h1, but the screen reader won't find it this way.
        x.setAttribute("role", "presentation");
    });

    // a user should be able to trigger the logo dialog by clicking on .jp-NotebookKernelLogo
    // the trusted status button needs alt and a clearer title .jp-NotebookTrustedStatus
    // the button.jp-Notebook-footer semantics are wrong. the button should be inside a footer, or role=contentinfo

    document.querySelectorAll("#main-panel").forEach((x,i) =>{x.setAttribute("aria-labelledby", "jp-title");});
    document.querySelectorAll("#menu-panel").forEach((x,i) =>{
        x.setAttribute("role", "banner"); x.setAttribute("aria-label", "Notebook Menu");
    });

## the main landmark containing a single notebook

this analysis is done for the less complicated notebook v7. in our primary main landmark we have only ONE edittable notebook.

In [17]:
    %%javascript
    document.querySelectorAll("#spacer-widget-top").forEach((x,i) =>{x.setAttribute("role", "separator");});
    document.querySelectorAll(".jp-Notebook").forEach((x,i) =>{x.setAttribute("aria-label", "Notebook Cells");});
    // role navigation -> banner, this is a landmark inside of main
    document.querySelectorAll(".jp-Toolbar").forEach((x,i) =>{
        x.setAttribute("role", "banner"); x.setAttribute("aria-label", "Notebook Actions");
    });    

<IPython.core.display.Javascript object>

## the notebook feed and cell articles

In [18]:
    %%javascript
    document.querySelectorAll(".jp-WindowedPanel-window").forEach((x,i) =>{
        x.setAttribute("role", "feed"); x.setAttribute("aria-label", "Cells");
    });
    var cells = document.querySelectorAll(".jp-Cell");
    cells.forEach((x,i) =>{
        x.setAttribute("role", "article"); 
        x.setAttribute("aria-posinset", i+1); 
        x.setAttribute("aria-setsize", cells.length); 
        var MD = x.className.includes("jp-MarkdownCell");
        var CODE = x.className.includes("jp-CodeCell");
        if (CODE){
            x.setAttribute("aria-label", `Code Cell ${i+1}`);
        } else if (MD){
            x.setAttribute("aria-label", `Markdown Cell ${i+1}`);
        } else {
            x.setAttribute("aria-label", `Cell ${i+1}`);
        };

        // i cant directly know the input number. semantically it is some kind of label.
        // this effort is concerned with the AOM, not WCAG. 
        // we do have a visible label so this should satisfy the visible label
        prompt = x.querySelectorAll(".jp-InputPrompt")[0];
        prompt.setAttribute("aria-hidden", "true");

        // STILL NEED TO ADD A LANDMARK FOR THE CELL
    });

<IPython.core.display.Javascript object>

### input area fixes

In [12]:
    %%javascript
    var cells = document.querySelectorAll(".jp-Cell");
    cells.forEach((x,i) =>{
        var MD = x.className.includes("jp-MarkdownCell");
        var CODE = x.className.includes("jp-CodeCell");

        // code, markdown, raw cells need different groupings.
        x.querySelectorAll(".jp-Cell-inputWrapper").forEach(
            (y)=>{
                y.setAttribute("role", "group");
                var label = prompt.textContent.slice(1, -2).trim();
                if (CODE){
                    if (label.length){
                        y.setAttribute("aria-label", `In ${label}`);
                    } else {
                        y.setAttribute("aria-label", `New Code Input`);
                    }
                } else if (MD){
                    y.setAttribute("aria-label", `Markdown Input`);
                } else {
                    y.setAttribute("aria-label", `Input`);
                }
            }
        );
        // make the cell toolbar an APG styled toolbar. this is the most cursory modification.
        x.querySelectorAll(".jp-cell-toolbar").forEach(
            (y)=>{
                // https://www.w3.org/WAI/ARIA/apg/patterns/toolbar/examples/toolbar/
                y.setAttribute("role", "toolbar");
                y.setAttribute("aria-label", `Cell ${i+1} Toolbar`);
                y.setAttribute("aria-controls", `cell-input-${i}`);
            }
        )
    });

<IPython.core.display.Javascript object>

### output area fixes

i am very not confident about the output semantics. my efforts were to understand the rendered cell's landmarks. the output semantics are pure presentation in reading mode, but editting mode is several HCI PhDs.

right now, i'm only comfortable calling the outputs a group, this means they may be announced by a screen reader to provide some relative position. on disk, in the notebook format, outputs are a list of structures, as are cells, and it might make the most sense to employ a nested feed pattern. when you start to thinking about async updates then ordering is out the window.

In [14]:
    %%javascript
    var cells = document.querySelectorAll(".jp-Cell");
    cells.forEach((x,i) =>{
        x.querySelectorAll(".jp-Cell-outputWrapper").forEach(
            (y)=>{
                // this will only be encountered on code cells.
                // i have no invested significant time into the semantics of the outputs
                // group is a fine semantic, but it is non specific.
                // increasingly i feel like the output is nested feed because it 
                // would allow outputs to be indexed. need more testing.
                y.setAttribute("role", "group");
                var label = prompt.textContent.slice(1, -2).trim();
                if (label.length){
                    y.setAttribute("aria-label", `Output ${label}`);
                } else {
                    // no need to worry about the output label when there is not execution
                    // aria hidden might be appropriate without testing on a screen reader.
                };
            }
        );
        x.querySelectorAll(".jp-OutputCollapser").forEach(
            (y)=>{
                // the collapser button has no semantics or visible label.
                // this feature is not accessible, but should because for folks using screen magnification.
            }
        )
    });

<IPython.core.display.Javascript object>

## conclusion

* the notebook and cell menu bars are probably fixed at the same time.
* the output area is where the research stops. it is less though out than the others. announcements are part of the equation too for the proper assistive experience.